## Importing Libraries

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

**1934. Confirmation Rate (Medium)**

**Table: Signups**

| Column Name    | Type     |
|----------------|----------|
| user_id        | int      |
| time_stamp     | datetime |

user_id is the column of unique values for this table.
Each row contains information about the signup time for the user with ID user_id.
 
**Table: Confirmations**

| Column Name    | Type     |
|----------------|----------|
| user_id        | int      |
| time_stamp     | datetime |
| action         | ENUM     |

(user_id, time_stamp) is the primary key (combination of columns with unique values) for this table.
user_id is a foreign key (reference column) to the Signups table.
action is an ENUM (category) of the type ('confirmed', 'timeout')
Each row of this table indicates that the user with ID user_id requested a confirmation message at time_stamp and that confirmation message was either confirmed ('confirmed') or expired without confirming ('timeout').
 
The confirmation rate of a user is the number of 'confirmed' messages divided by the total number of requested confirmation messages. The confirmation rate of a user that did not request any confirmation messages is 0. Round the confirmation rate to two decimal places.

**Write a solution to find the confirmation rate of each user.**

Return the result table in any order.

The result format is in the following example.

**Example 1:**

**Input:**
**Signups table:**
| user_id | time_stamp          |
|---------|---------------------|
| 3       | 2020-03-21 10:16:13 |
| 7       | 2020-01-04 13:57:59 |
| 2       | 2020-07-29 23:09:44 |
| 6       | 2020-12-09 10:39:37 |

**Confirmations table:**
| user_id | time_stamp          | action    |
|---------|---------------------|-----------|
| 3       | 2021-01-06 03:30:46 | timeout   |
| 3       | 2021-07-14 14:00:00 | timeout   |
| 7       | 2021-06-12 11:57:29 | confirmed |
| 7       | 2021-06-13 12:58:28 | confirmed |
| 7       | 2021-06-14 13:59:27 | confirmed |
| 2       | 2021-01-22 00:00:00 | confirmed |
| 2       | 2021-02-28 23:59:59 | timeout   |

**Output:**
| user_id | confirmation_rate |
|---------|-------------------|
| 6       | 0.00              |
| 3       | 0.00              |
| 7       | 1.00              |
| 2       | 0.50              |

**Explanation:**
- User 6 did not request any confirmation messages. The confirmation rate is 0.
- User 3 made 2 requests and both timed out. The confirmation rate is 0.
- User 7 made 3 requests and all were confirmed. The confirmation rate is 1.
- User 2 made 2 requests where one was confirmed and the other timed out. The confirmation rate is 1 / 2 = 0.5.

In [0]:
signups_data_1934 = [
    (3, "2020-03-21 10:16:13"),
    (7, "2020-01-04 13:57:59"),
    (2, "2020-07-29 23:09:44"),
    (6, "2020-12-09 10:39:37")
]
signups_columns_1934 = ["user_id", "time_stamp"]
signups_df_1934 = spark.createDataFrame(signups_data_1934, signups_columns_1934)
signups_df_1934.show()

confirmations_data_1934 = [
    (3, "2021-01-06 03:30:46", "timeout"),
    (3, "2021-07-14 14:00:00", "timeout"),
    (7, "2021-06-12 11:57:29", "confirmed"),
    (7, "2021-06-13 12:58:28", "confirmed"),
    (7, "2021-06-14 13:59:27", "confirmed"),
    (2, "2021-01-22 00:00:00", "confirmed"),
    (2, "2021-02-28 23:59:59", "timeout")
]

confirmations_columns_1934 = ["user_id", "time_stamp", "action"]
confirmations_df_1934 = spark.createDataFrame(confirmations_data_1934, confirmations_columns_1934)
confirmations_df_1934.show()

+-------+-------------------+
|user_id|         time_stamp|
+-------+-------------------+
|      3|2020-03-21 10:16:13|
|      7|2020-01-04 13:57:59|
|      2|2020-07-29 23:09:44|
|      6|2020-12-09 10:39:37|
+-------+-------------------+

+-------+-------------------+---------+
|user_id|         time_stamp|   action|
+-------+-------------------+---------+
|      3|2021-01-06 03:30:46|  timeout|
|      3|2021-07-14 14:00:00|  timeout|
|      7|2021-06-12 11:57:29|confirmed|
|      7|2021-06-13 12:58:28|confirmed|
|      7|2021-06-14 13:59:27|confirmed|
|      2|2021-01-22 00:00:00|confirmed|
|      2|2021-02-28 23:59:59|  timeout|
+-------+-------------------+---------+



In [0]:
confirmations_agg_df_1934 = confirmations_df_1934\
                            .groupBy("user_id")\
                                .agg(
                                    count("*").alias("total_requests"),
                                    sum(when(col("action") == "confirmed", 1).otherwise(0)).alias("confirmed_requests")
                                )

In [0]:
signups_df_1934\
    .join(
        confirmations_agg_df_1934, on="user_id", how="left")\
            .withColumn(
                "confirmation_rate",
                when(col("total_requests").isNull(), 0.0)
                .otherwise(col("confirmed_requests") / col("total_requests"))
                )\
                    .select("user_id",round("confirmation_rate", 2).alias("confirmation_rate")
                            ).show()

+-------+-----------------+
|user_id|confirmation_rate|
+-------+-----------------+
|      3|              0.0|
|      7|              1.0|
|      2|              0.5|
|      6|              0.0|
+-------+-----------------+

